In [2]:
from PIL import Image

from resizeimage import resizeimage
import cv2
import numpy as np

img = Image.open("rot.jpg")
im2 = img.convert('RGBA')
#size = jpgfile.size[0]/5 , jpgfile.size[1]/5

#cover = resizeimage.resize_cover(jpgfile, size)

#cover.show()
x = 0
y = 0
for i in range(0,361):
#a reviser
    rot = im2.rotate(i, expand = True)
    fff = Image.new('RGBA', rot.size, (255,)*4)
    out = Image.composite(rot, fff, rot)
    out.convert(img.mode).save('out1.jpg')
    image = cv2.imread("out1.jpg")
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV)
#cv2.imshow('binary',thresh)
#cv2.waitKey(0)

    thresh_sum = thresh.sum(axis=1)
#thresh_sum.show

    a = []

    a = np.append(a,thresh_sum)
    variance = 0
    variance = np.var(thresh_sum,0)
    print (i,variance)
    if variance > x :
        x = variance
        y = i
    out.convert(img.mode).save('out1.jpg')
    image = cv2.imread("out1.jpg")
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV)
print(x,i)

0 27758600987.452675
1 25889569781.921265
2 24187395757.018246
3 22661906010.958958
4 21237257224.80032
5 19954129016.592144
6 18786937591.897415
7 17678445236.960876
8 16649194889.176699
9 15699507514.857286
10 14787600831.51463
11 13899768258.296684
12 13093866968.186724
13 12297427985.537693
14 11514381313.974726
15 10777814038.740648
16 10080344109.46745
17 9376895083.61005
18 8717734107.247501
19 8107934884.974624
20 7538364454.195994
21 6996346370.043214
22 6530390167.943806
23 6140327871.992792
24 5790268281.394528
25 5518861154.282221
26 5301669238.330118
27 5175064736.648445
28 5126316720.332044
29 5181744055.426164
30 5323408955.942387
31 5568951709.214012
32 5921471315.146987
33 6336241085.92134
34 6779306869.343318
35 7260283442.674896
36 7735165270.046921
37 8212267102.381954
38 8673604969.099823
39 9104970077.97663
40 9478440542.692564
41 9807781182.353981
42 10060456102.691017
43 10247928787.4807
44 10351932817.0324
45 10417454831.62545
46 10448123642.732363
47 104485915